# Computer Vision Homework 3: Big vs Small Models

## Brief

Due date: Nov 13, 2023

Required files: `homework-3.ipynb`, `report.pdf`

To download the jupyter notebook from colab, you can refer to the colab tutorial we gave.


## Codes for Problem 1 and Problem 2

### Import Packages

In [ ]:
import glob
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from PIL import Image
from torch.utils.data import DataLoader, Dataset, RandomSampler
from torchvision import transforms, models, datasets
from tqdm import tqdm

%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Mounted at /content/drive


### Check GPU Environment

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')

Using cuda device


In [ ]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-fa370ab3-67df-ac5c-8fa4-ce53f605f39d)


### Set the Seed to Reproduce the Result

In [ ]:
def set_all_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
set_all_seed(123)

### Create Dataset and Dataloader

In [ ]:
batch_size = 256

mean = (0.4914, 0.4822, 0.4465)
std = (0.2471, 0.2435, 0.2616)
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_dataset = datasets.CIFAR10(root='data', train=True, download=True, transform=train_transform)
valid_dataset = datasets.CIFAR10(root='data', train=False, download=True, transform=test_transform)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

sixteenth_train_sampler = RandomSampler(train_dataset, num_samples=len(train_dataset)//16)
half_train_sampler = RandomSampler(train_dataset, num_samples=len(train_dataset)//2)

sixteenth_train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sixteenth_train_sampler)
half_train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=half_train_sampler)

100%|██████████| 170498071/170498071 [00:01<00:00, 86495185.29it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


### Load Models

In [ ]:
# HINT: Remember to change the model to 'resnet50' and the weights to weights="IMAGENET1K_V1" when needed.
model_resnet18_none = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights=None)
model_resnet50_none = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights=None)
model_resnet18_weight = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights='ResNet18_Weights.IMAGENET1K_V1')
model_resnet50_weight = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights='ResNet50_Weights.IMAGENET1K_V1')

# Background: The original resnet18 is designed for ImageNet dataset to predict 1000 classes.
# TODO: Change the output of the model to 10 class.
model_resnet18_none.fc = nn.Linear(in_features=512, out_features=10, bias=True)
model_resnet18_none = model_resnet18_none.to(device)
model_resnet50_none.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
model_resnet50_none = model_resnet50_none.to(device)
model_resnet18_weight.fc = nn.Linear(in_features=512, out_features=10, bias=True)
model_resnet18_weight = model_resnet18_weight.to(device)
model_resnet50_weight.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
model_resnet50_weight = model_resnet50_weight.to(device)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 88.5MB/s]
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 119MB/s]


### Training and Testing Models

In [ ]:
# TODO: Fill in the code cell according to the pytorch tutorial we gave.
def train(dataloader, model, loss_fn, optimizer):
    num_batches = len(dataloader)
    size = len(dataloader.sampler)
    epoch_loss = 0
    correct = 0

    model.train()

    for X, y in tqdm(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        pred = pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(y.view_as(pred)).sum().item()

    avg_epoch_loss = epoch_loss / num_batches
    avg_acc = correct / size

    return avg_epoch_loss, avg_acc

In [ ]:
def test(dataloader, model, loss_fn):

  num_batches = len(dataloader)
  size = len(dataloader.sampler)
  epoch_loss = 0
  correct = 0

  model.eval()

  with torch.no_grad():
    for X, y in tqdm(dataloader):
      X, y = X.to(device), y.to(device)

      pred = model(X)

      epoch_loss += loss_fn(pred, y).item()
      pred = pred.argmax(dim=1, keepdim=True)
      correct += pred.eq(y.view_as(pred)).sum().item()


  avg_epoch_loss = epoch_loss / num_batches
  avg_acc = correct / size

  return avg_epoch_loss, avg_acc

In [ ]:
def plot_loss_curve(data, models_name, dataloader):
    plt.figure()
    plt.plot(data.index, data['Train_Loss'], label='Train Loss')
    plt.plot(data.index, data['Test_Loss'], label='Test Loss')
    plt.legend(loc='upper right')
    plt.xlabel('Episodes')
    plt.ylabel('Loss')
    plt.title('Loss curve')
    plt.savefig(f'{models_name}_{len(dataloader.dataset)//len(dataloader.sampler)}_Loss_curve.png')
    plt.close()
    # Download the loss curve plot
    files.download(f'{models_name}_{len(dataloader.dataset)//len(dataloader.sampler)}_Loss_curve.png')

def plot_accuracy_curve(data, models_name, dataloader):
    plt.figure()
    plt.plot(data.index, data['Train_Acc'], label='Train Accuracy')
    plt.plot(data.index, data['Test_Acc'], label='Test Accuracy')
    plt.legend(loc='upper left')
    plt.xlabel('Episodes')
    plt.ylabel('Accuracy')
    plt.title('Accuracy curve')
    plt.savefig(f'{models_name}_{len(dataloader.dataset)//len(dataloader.sampler)}_Accuracy_curve.png')
    plt.close()
    # Download the accuracy curve plot
    files.download(f'{models_name}_{len(dataloader.dataset)//len(dataloader.sampler)}_Accuracy_curve.png')

In [ ]:
epochs = 50
models = [model_resnet18_none]
model_names = ['resnet18_none']
dataloaders = [sixteenth_train_dataloader, half_train_dataloader, train_dataloader]
dataloader_sizes = [1/16, 1/2, 1]
best_test_acc = 0

for model, model_name in zip(models, model_names): # 每個 model 輪流訓練
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

  for dataloader, dataloader_size in zip(dataloaders, dataloader_sizes): # 每個datasize選擇
    print(f'Model: {model_name}')
    print(f'Dataloader\'s sampler: {len(dataloader.dataset)//len(dataloader.sampler)}')

    data = pd.DataFrame(columns=['Train_Loss', 'Train_Acc', 'Test_Loss', 'Test_Acc'])

    for epoch in range(epochs): # 每回合訓練
      print(f"\n Epoch {epoch + 1:2d}:")
      print('Train loading:')
      train_loss, train_acc = train(dataloader, model, loss_fn, optimizer)
      print('Test loading:')
      test_loss, test_acc = test(valid_dataloader, model, loss_fn)
      print(f"\n Train_Loss = {train_loss:.4f}, Train_Acc = {train_acc:.2f} \n Test_Loss  = {test_loss:.4f}, Test_Acc  = {test_acc:.2f}")
      if test_acc > best_test_acc:
        best_test_acc = test_acc
      data = pd.concat([data, pd.DataFrame.from_records([{'Train_Loss': train_loss, 'Train_Acc': train_acc, 'Test_Loss': test_loss, 'Test_Acc': test_acc}])], ignore_index=True)

    train_acc_key = f'{model_name}_{dataloader_size}_Train_Acc'
    test_acc_key = f'{model_name}_{dataloader_size}_Test_Acc'
    locals()[train_acc_key] = train_acc
    locals()[test_acc_key] = test_acc

    plot_loss_curve(data, model_name, dataloader) # 繪製loss_curve圖
    plot_accuracy_curve(data, model_name, dataloader) # 繪製accuracy_curve圖
    print(f'best_test_acc={best_test_acc:.2f}')


    print("\n Done!")

Model: resnet18_none
Dataloader's sampler: 16

 Epoch  1:
Train loading:


100%|██████████| 13/13 [00:10<00:00,  1.29it/s]


Test loading:


100%|██████████| 40/40 [00:03<00:00, 11.78it/s]



 Train_Loss = 2.1446, Train_Acc = 0.25 
 Test_Loss  = 2.4059, Test_Acc  = 0.16

 Epoch  2:
Train loading:


...

Test loading:


100%|██████████| 40/40 [00:03<00:00, 12.02it/s]



 Train_Loss = 0.2463, Train_Acc = 0.91 
 Test_Loss  = 0.5852, Test_Acc  = 0.83

 Epoch 76:
Train loading:


 81%|████████  | 79/98 [00:13<00:03,  5.94it/s]


KeyboardInterrupt: ignored

In [ ]:
epochs = 50
models = [model_resnet50_none]
model_names = ['resnet50_none']
dataloaders = [sixteenth_train_dataloader, half_train_dataloader, train_dataloader]
dataloader_sizes = [1/16, 1/2, 1]
best_test_acc = 0
for model, model_name in zip(models, model_names): # 每個 model 輪流訓練
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

  for dataloader, dataloader_size in zip(dataloaders, dataloader_sizes): # 每個datasize選擇
    print(f'Model: {model_name}')
    print(f'Dataloader\'s sampler: {len(dataloader.dataset)//len(dataloader.sampler)}')

    data = pd.DataFrame(columns=['Train_Loss', 'Train_Acc', 'Test_Loss', 'Test_Acc'])

    for epoch in range(epochs): # 每回合訓練
      print(f"\n Epoch {epoch + 1:2d}:")
      print('Train loading:')
      train_loss, train_acc = train(dataloader, model, loss_fn, optimizer)
      print('Test loading:')
      test_loss, test_acc = test(valid_dataloader, model, loss_fn)
      print(f"\n Train_Loss = {train_loss:.4f}, Train_Acc = {train_acc:.2f} \n Test_Loss  = {test_loss:.4f}, Test_Acc  = {test_acc:.2f}")
      if test_acc > best_test_acc:
        best_test_acc = test_acc
      data = pd.concat([data, pd.DataFrame.from_records([{'Train_Loss': train_loss, 'Train_Acc': train_acc, 'Test_Loss': test_loss, 'Test_Acc': test_acc}])], ignore_index=True)

    train_acc_key = f'{model_name}_{dataloader_size}_Train_Acc'
    test_acc_key = f'{model_name}_{dataloader_size}_Test_Acc'
    locals()[train_acc_key] = train_acc
    locals()[test_acc_key] = test_acc

    plot_loss_curve(data, model_name, dataloader) # 繪製loss_curve圖
    plot_accuracy_curve(data, model_name, dataloader) # 繪製accuracy_curve圖
    print(f'best_test_acc={best_test_acc:.2f}')

    print("\n Done!")

Model: resnet50_none
Dataloader's sampler: 16
Train loading:


100%|██████████| 13/13 [00:02<00:00,  4.48it/s]


Test loading:


100%|██████████| 40/40 [00:04<00:00,  8.40it/s]



 Epoch  1:
 Train_Loss = 2.8028, Train_Acc = 0.15 
 Test_Loss  = 2.3908, Test_Acc  = 0.10
Train loading:


...


 Epoch 49:
 Train_Loss = 0.2928, Train_Acc = 0.90 
 Test_Loss  = 0.5690, Test_Acc  = 0.83
Train loading:


100%|██████████| 196/196 [00:42<00:00,  4.56it/s]


Test loading:


100%|██████████| 40/40 [00:03<00:00, 11.08it/s]



 Epoch 50:
 Train_Loss = 0.2878, Train_Acc = 0.90 
 Test_Loss  = 0.5232, Test_Acc  = 0.84

 Done!


In [ ]:
epochs = 50
models = [model_resnet18_weight]
model_names = ['resnet18_weight']
dataloaders = [sixteenth_train_dataloader, half_train_dataloader, train_dataloader]
dataloader_sizes = [1/16, 1/2, 1]
best_test_acc = 0
for model, model_name in zip(models, model_names): # 每個 model 輪流訓練
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

  for dataloader, dataloader_size in zip(dataloaders, dataloader_sizes): # 每個datasize選擇
    print(f'Model: {model_name}')
    print(f'Dataloader\'s sampler: {len(dataloader.dataset)//len(dataloader.sampler)}')

    data = pd.DataFrame(columns=['Train_Loss', 'Train_Acc', 'Test_Loss', 'Test_Acc'])

    for epoch in range(epochs): # 每回合訓練
      print(f"\n Epoch {epoch + 1:2d}:")
      print('Train loading:')
      train_loss, train_acc = train(dataloader, model, loss_fn, optimizer)
      print('Test loading:')
      test_loss, test_acc = test(valid_dataloader, model, loss_fn)
      print(f"\n Train_Loss = {train_loss:.4f}, Train_Acc = {train_acc:.2f} \n Test_Loss  = {test_loss:.4f}, Test_Acc  = {test_acc:.2f}")
      if test_acc > best_test_acc:
        best_test_acc = test_acc
      data = pd.concat([data, pd.DataFrame.from_records([{'Train_Loss': train_loss, 'Train_Acc': train_acc, 'Test_Loss': test_loss, 'Test_Acc': test_acc}])], ignore_index=True)

    train_acc_key = f'{model_name}_{dataloader_size}_Train_Acc'
    test_acc_key = f'{model_name}_{dataloader_size}_Test_Acc'
    locals()[train_acc_key] = train_acc
    locals()[test_acc_key] = test_acc

    plot_loss_curve(data, model_name, dataloader) # 繪製loss_curve圖
    plot_accuracy_curve(data, model_name, dataloader) # 繪製accuracy_curve圖
    print(f'best_test_acc={best_test_acc:.2f}')

    print("\n Done!")

Model: resnet18_weight
Dataloader's sampler: 16
Train loading:


100%|██████████| 13/13 [00:01<00:00,  6.75it/s]


Test loading:


100%|██████████| 40/40 [00:03<00:00, 12.60it/s]



 Epoch  1:
 Train_Loss = 1.8597, Train_Acc = 0.36 
 Test_Loss  = 1.6078, Test_Acc  = 0.44
Train loading:


...


 Epoch 49:
 Train_Loss = 0.0858, Train_Acc = 0.97 
 Test_Loss  = 0.6252, Test_Acc  = 0.86
Train loading:


100%|██████████| 196/196 [00:34<00:00,  5.76it/s]


Test loading:


100%|██████████| 40/40 [00:03<00:00, 12.55it/s]



 Epoch 50:
 Train_Loss = 0.1053, Train_Acc = 0.96 
 Test_Loss  = 0.5805, Test_Acc  = 0.86

 Done!


In [ ]:
epochs = 50
models = [model_resnet50_weight]
model_names = ['resnet50_weight']
dataloaders = [sixteenth_train_dataloader, half_train_dataloader, train_dataloader]
dataloader_sizes = [1/16, 1/2, 1]
best_test_acc = 0
for model, model_name in zip(models, model_names): # 每個 model 輪流訓練
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

  for dataloader, dataloader_size in zip(dataloaders, dataloader_sizes): # 每個datasize選擇
    print(f'Model: {model_name}')
    print(f'Dataloader\'s sampler: {len(dataloader.dataset)//len(dataloader.sampler)}')

    data = pd.DataFrame(columns=['Train_Loss', 'Train_Acc', 'Test_Loss', 'Test_Acc'])

    for epoch in range(epochs): # 每回合訓練
      print(f"\n Epoch {epoch + 1:2d}:")
      print('Train loading:')
      train_loss, train_acc = train(dataloader, model, loss_fn, optimizer)
      print('Test loading:')
      test_loss, test_acc = test(valid_dataloader, model, loss_fn)
      print(f"\n Train_Loss = {train_loss:.4f}, Train_Acc = {train_acc:.2f} \n Test_Loss  = {test_loss:.4f}, Test_Acc  = {test_acc:.2f}")
      if test_acc > best_test_acc:
        best_test_acc = test_acc
      data = pd.concat([data, pd.DataFrame.from_records([{'Train_Loss': train_loss, 'Train_Acc': train_acc, 'Test_Loss': test_loss, 'Test_Acc': test_acc}])], ignore_index=True)

    train_acc_key = f'{model_name}_{dataloader_size}_Train_Acc'
    test_acc_key = f'{model_name}_{dataloader_size}_Test_Acc'
    locals()[train_acc_key] = train_acc
    locals()[test_acc_key] = test_acc

    plot_loss_curve(data, model_name, dataloader) # 繪製loss_curve圖
    plot_accuracy_curve(data, model_name, dataloader) # 繪製accuracy_curve圖
    print(f'best_test_acc={best_test_acc:.2f}')

    print("\n Done!")

Model: resnet50_weight
Dataloader's sampler: 16

 Epoch  1:
Train loading:


 92%|█████████▏| 12/13 [00:02<00:00,  5.34it/s]

In [ ]:
# 繪製整體比較圖
# none weights
plt.figure()
for model_name in ['resnet18_none', 'resnet50_none']:
  train_acc_values = [locals()[f'{model_name}_0.0625_Train_Acc'], locals()[f'{model_name}_0.5_Train_Acc'], locals()[f'{model_name}_1_Train_Acc']]
  plt.plot(dataloader_sizes, train_acc_values, label=model_name)
plt.legend(['resnet18', 'resnet50'], loc='upper left')
plt.xlabel('Dataset Size')
plt.ylabel('Accuracy')
plt.title('Dataset Size vs Accuracy, weights none')
plt.savefig(f'Dataset Size vs Accuracy, weights none.png') # 儲存路徑/content/drive/MyDrive/Colab Notebooks/自己的資料夾/
plt.close()
files.download(f'Dataset Size vs Accuracy, weights none.png')
# initialized weights
plt.figure()
for model_name in ['resnet18_weight', 'resnet50_weight']:
  train_acc_values = [locals()[f'{model_name}_0.0625_Train_Acc'], locals()[f'{model_name}_0.5_Train_Acc'], locals()[f'{model_name}_1_Train_Acc']]
  plt.plot(dataloader_sizes, train_acc_values, label=model_name)
plt.legend(['resnet18 with initialized weights', 'resnet50 with initialized weights'], loc='upper left')
plt.xlabel('Dataset Size')
plt.ylabel('Accuracy')
plt.title('Dataset Size vs Accuracy, initialized weights')
plt.savefig(f'Dataset Size vs Accuracy, initialized weights.png') # 儲存路徑/content/drive/MyDrive/Colab Notebooks/自己的資料夾/
plt.close()
files.download(f'Dataset Size vs Accuracy, initialized weights.png')

## Codes for Problem 3

In [ ]:
# TODO: Try to achieve the best performance given all training data using whatever model and training strategy.
# (New) (You cannot use the model that was pretrained on CIFAR10)
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 128, 5, padding=2)
    self.conv2 = nn.Conv2d(128, 128, 5, padding=2)
    self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
    self.conv4 = nn.Conv2d(256, 256, 3, padding=1)
    self.pool = nn.MaxPool2d(2, 2)
    self.bn_conv1 = nn.BatchNorm2d(128)
    self.bn_conv2 = nn.BatchNorm2d(128)
    self.bn_conv3 = nn.BatchNorm2d(256)
    self.bn_conv4 = nn.BatchNorm2d(256)
    self.bn_dense1 = nn.BatchNorm1d(1024)
    self.bn_dense2 = nn.BatchNorm1d(512)
    self.dropout_conv = nn.Dropout2d(p=0.25)
    self.dropout = nn.Dropout(p=0.5)
    self.fc1 = nn.Linear(256 * 8 * 8, 1024)
    self.fc2 = nn.Linear(1024, 512)
    self.fc3 = nn.Linear(512, 10)

  def conv_layers(self, x):
    out = F.relu(self.bn_conv1(self.conv1(x)))
    out = F.relu(self.bn_conv2(self.conv2(out)))
    out = self.pool(out)
    out = self.dropout_conv(out)
    out = F.relu(self.bn_conv3(self.conv3(out)))
    out = F.relu(self.bn_conv4(self.conv4(out)))
    out = self.pool(out)
    out = self.dropout_conv(out)
    return out

  def dense_layers(self, x):
    out = F.relu(self.bn_dense1(self.fc1(x)))
    out = self.dropout(out)
    out = F.relu(self.bn_dense2(self.fc2(out)))
    out = self.dropout(out)
    out = self.fc3(out)
    return out

  def forward(self, x):
    out = self.conv_layers(x)
    out = out.view(-1, 256 * 8 * 8)
    out = self.dense_layers(out)
    return out

In [ ]:
Net = Net().to(torch.device('cuda'))
epochs = 50
models = [Net]
model_names = ['Net']
dataloaders = [train_dataloader]
best_test_acc = 0
for model, model_name in zip(models, model_names):
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)
  for dataloader in (dataloaders):
    data = pd.DataFrame(columns=['Train_Loss', 'Train_Acc', 'Test_Loss', 'Test_Acc'])
    for epoch in range(epochs):
      print(f"\n Epoch {epoch + 1:2d}:")
      print('Train loading:')
      train_loss, train_acc = train(dataloader, model, loss_fn, optimizer)
      print('Test loading:')
      test_loss, test_acc = test(valid_dataloader, model, loss_fn)
      print(f"\n Train_Loss = {train_loss:.4f}, Train_Acc = {train_acc:.2f} \n Test_Loss  = {test_loss:.4f}, Test_Acc  = {test_acc:.2f}")
      if test_acc > best_test_acc:
        best_test_acc = test_acc
      data = pd.concat([data, pd.DataFrame.from_records([{'Train_Loss': train_loss, 'Train_Acc': train_acc, 'Test_Loss': test_loss, 'Test_Acc': test_acc}])], ignore_index=True)

    plot_loss_curve(data, model_name, dataloader) # 繪製loss_curve圖
    plot_accuracy_curve(data, model_name, dataloader) # 繪製accuracy_curve圖
    print(f'best_test_acc={best_test_acc:.2f}')

    print("\n Done!")


 Epoch  1:
Train loading:


100%|██████████| 196/196 [00:53<00:00,  3.70it/s]


Test loading:


100%|██████████| 40/40 [00:04<00:00,  8.11it/s]



 Train_Loss = 1.6507, Train_Acc = 0.39 
 Test_Loss  = 1.2707, Test_Acc  = 0.54

 Epoch  2:
Train loading:


...

100%|██████████| 196/196 [00:52<00:00,  3.77it/s]


Test loading:


100%|██████████| 40/40 [00:04<00:00,  9.44it/s]



 Train_Loss = 1.1029, Train_Acc = 0.60 
 Test_Loss  = 0.8336, Test_Acc  = 0.71

 Epoch  4:
Train loading:


 34%|███▍      | 67/196 [00:17<00:33,  3.88it/s]

## Problems

1. (30%) Finish the rest of the codes for Problem 1 and Problem 2 according to the hint. (2 code cells in total.)
2. Train small model (resnet18) and big model (resnet50) from scratch on `sixteenth_train_dataloader`, `half_train_dataloader`, and `train_dataloader` respectively.
3. (30%) Achieve the best performance given all training data using whatever model and training strategy.  
  (You cannot use the model that was pretrained on CIFAR10)



## Discussion

Write down your insights in the report. The file name should be report.pdf.
For the following discussion, please present the results graphically as shown in Fig. 1 and discuss them.

- (30%) The relationship between the accuracy, model size, and the training dataset size.  
    (Total 6 models. Small model trains on the sixteenth, half, and all data. Big model trains on the sixteenth, half, and all data. If the result is different from Fig.1, please explain the possible reasons.)
- (10%) What if we train the ResNet with ImageNet initialized weights (`weights="IMAGENET1K_V1"`).
Please explain why the relationship changed this way?

Hint: You can try different hyperparameters combinations when training the models.

## Credits

1. [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html)